In [123]:
from __future__ import absolute_import
from __future__ import print_function

import os
import time
import numpy as np
import keras.backend as K
import argparse
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Input, Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.regularizers import l2
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.metrics import AUC
import numpy as np
from keras.callbacks import LearningRateScheduler
import cv2
from imblearn.datasets import make_imbalance
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score


In [124]:
alpha = 1.0
beta = 1.0
noise_ratio=20
epochs=1
dataset='mnist'
model_name='sl'
batch_size=128
dasm= False
asym=False

In [125]:
# prepare folders
folders = ['data', 'model', 'log']
for folder in folders:
    path = os.path.join('./', folder)
    if not os.path.exists(path):
        os.makedirs(path)

In [126]:
(X_train, y_train_1), (X_test, y_test) = mnist.load_data()

In [ ]:

if dasm:
  X_train = X_train.reshape(-1,784)
  X_train, y_tr = make_imbalance(X_train, y_train_1,
                                sampling_strategy={0: 2550, 1: 6675, 2: 3280, 3: 6100, 4: 3850, 5: 5400, 6: 3065, 7: 5860, 8: 5000, 9: 3654,},
                                random_state=42)
else:
  y_tr=y_train_1
  


In [128]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train / 255.0
X_test = X_test / 255.0

In [129]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [130]:
y_train_clean = np.copy(y_tr)

In [131]:
def other_class(n_classes, current_class):
    if current_class < 0 or current_class >= n_classes:
        error_str = "class_ind must be within the range (0, nb_classes - 1)"
        raise ValueError(error_str)

    other_class_list = list(range(n_classes))
    other_class_list.remove(current_class)
    other_class = np.random.choice(other_class_list)
    return other_class

In [ ]:
# y_train_clean = np.copy(y_train)
if noise_ratio > 0:
    if asym:
            data_file = "data/asym_%s_train_labels_%s.npy" % (dataset, noise_ratio)
    else:
            data_file = "data/%s_train_labels_%s.npy" % (dataset, noise_ratio)
    if os.path.isfile(data_file):
            y_tr = np.load(data_file)
    else:
            if asym:
                if dataset == 'mnist':
                    # 1 < - 7, 2 -> 7, 3 -> 8, 5 <-> 6
                    source_class = [7, 2, 3, 5, 6]
                    target_class = [1, 7, 8, 6, 5]
                if dataset == 'mnist' :
                    for s, t in zip(source_class, target_class):
                        cls_idx = np.where(y_train_clean == s)[0]
                        n_noisy = int(noise_ratio * cls_idx.shape[0] / 100)
                        noisy_sample_index = np.random.choice(cls_idx, n_noisy, replace=False)
                        y_tr[noisy_sample_index] = t
                        
            else:
                n_samples = y_tr.shape[0]
                n_noisy = int(noise_ratio * n_samples / 100)
                class_index = [np.where(y_train_clean == i)[0] for i in range(10)]
                class_noisy = int(n_noisy / 10)

                noisy_idx = []
                for d in range(10):
                    noisy_class_index = np.random.choice(class_index[d], class_noisy, replace=False)
                    noisy_idx.extend(noisy_class_index)

                for i in noisy_idx:
                    y_tr[i] = other_class(n_classes=10, current_class=y_tr[i])
            np.save(data_file, y_tr)

    print("Print noisy label generation statistics:")
    for i in range(10):
            n_noisy = np.sum(y_tr == i)
            print("Noisy class %s, has %s samples." % (i, n_noisy))


y_train_clean = to_categorical(y_train_clean, 10)
y_tr = to_categorical(y_tr, 10)
y_test =to_categorical(y_test, 10)

print("y_train_clean:", y_train_clean.shape)
print("X_train:", X_train.shape)
print("y_train:", y_tr.shape)
print("X_test:", X_test.shape)
print("y_test", y_test.shape)

In [ ]:
n_images = X_train.shape[0]
image_shape = X_train.shape[1:]
num_classes = y_tr.shape[1]
print("n_images", n_images, "num_classes", num_classes, "image_shape:", image_shape)

In [134]:
# define P for forward and backward loss
P = np.eye(num_classes)

In [135]:
input_shape=image_shape
img_input = Input(shape=input_shape)



In [136]:
x = Conv2D(32, (3, 3), padding='same', kernel_initializer="he_normal", name='conv1')(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

x = Conv2D(64, (3, 3), padding='same', kernel_initializer="he_normal", name='conv2')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)

x = Flatten()(x)

x = Dense(128, kernel_initializer="he_normal", name='fc1')(x)
x = BatchNormalization()(x)
x = Activation('relu', name='lid')(x)
# x = Dropout(0.2)(x)

x = Dense(num_classes, kernel_initializer="he_normal")(x)
x = Activation(tf.nn.softmax)(x)

model = Model(img_input, x)

In [ ]:
optimizer = SGD(lr=0.1, decay=1e-4, momentum=0.9)

In [138]:
def symmetric_cross_entropy(alpha, beta):
    def loss(y_true, y_pred):
        y_true_1 = y_true
        y_pred_1 = y_pred

        y_true_2 = y_true
        y_pred_2 = y_pred

        y_pred_1 = tf.clip_by_value(y_pred_1, 1e-7, 1.0)
        y_true_2 = tf.clip_by_value(y_true_2, 1e-4, 1.0)
        # print(y_pred.numpy())
        return alpha*tf.reduce_mean(-tf.reduce_sum(y_true_1 * tf.math.log(y_pred_1), axis = -1)) + beta*tf.reduce_mean(-tf.reduce_sum(y_pred_2 * tf.math.log(y_true_2), axis = -1))
    return loss

In [139]:
def symmetric_cross_entropy(alpha, beta):
    def loss(y_true, y_pred):
        y_true_1 = y_true
        y_pred_1 = y_pred

        y_true_2 = y_true
        y_pred_2 = y_pred

        y_pred_1 = tf.clip_by_value(y_pred_1, 1e-7, 1.0)
        y_true_2 = tf.clip_by_value(y_true_2, 1e-4, 1.0)

        return alpha*tf.reduce_mean(-tf.reduce_sum(y_true_1 * tf.math.log(y_pred_1), axis = -1)) + beta*tf.reduce_mean(-tf.reduce_sum(y_pred_2 * tf.math.log(y_true_2), axis = -1))
    return loss

In [140]:

loss = symmetric_cross_entropy(alpha,beta)


In [141]:
#################################################################################################
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [142]:
# model ##########################################################
model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['accuracy',f1_m,precision_m, recall_m,AUC()],
    
)

In [143]:
# asym=False
if asym:
    model_save_file = "model/asym_%s_%s_%s.{epoch:02d}.hdf5" % ('sl','MNIST', 0)
else:
    model_save_file = "model/%s_%s_%s.{epoch:02d}.hdf5" % ('sl','MNIST', 0)

In [144]:
## do real-time updates using callbakcs
callbacks = []

In [ ]:
cp_callback = ModelCheckpoint(model_save_file,
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=False,
                                save_weights_only=True,
                                period=1)
callbacks.append(cp_callback)

In [146]:
# Set random seed
np.random.seed(123)
def get_lr_scheduler(dataset):
    if dataset in ['mnist']:
        def scheduler(epoch):
            if epoch > 30:
                return 0.001
            elif epoch > 10:
                return 0.01
            else:
                return 0.1
        return LearningRateScheduler(scheduler)

In [147]:


lr_scheduler = get_lr_scheduler('mnist')
callbacks.append(lr_scheduler)

In [148]:

class LoggerCallback(Callback):
    """
    Log train/val loss and acc into file for later plots.
    """
    def __init__(self, model, X_train, y_tr, y_train_clean, X_test, y_test, dataset,
                 model_name, noise_ratio, asym, epochs, alpha, beta):
        super(LoggerCallback, self).__init__()
        self.model = model
        self.X_train = X_train
        self.y_tr = y_tr
        self.y_train_clean = y_train_clean
        self.X_test = X_test
        self.y_test = y_test
        self.n_class = y_tr.shape[1]
        self.dataset = dataset
        self.model_name = model_name
        self.noise_ratio = noise_ratio
        self.asym = asym
        self.epochs = epochs
        self.alpha = alpha
        self.beta = beta

        self.train_loss = []
        self.test_loss = []
        self.train_acc = []
        self.test_acc = []
        self.train_loss_class = [None]*self.n_class
        self.train_acc_class = [None]*self.n_class

        # the followings are used to estimate LID
        self.lid_k = 20
        self.lid_subset = 128
        self.lids = []

        # complexity - Critical Sample Ratio (csr)
        self.csr_subset = 500
        self.csr_batchsize = 100
        self.csrs = []

    def on_epoch_end(self, epoch, logs={}):
        tr_acc = logs.get('acc')
        tr_loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        val_acc = logs.get('val_acc')

        self.train_loss.append(tr_loss)
        self.test_loss.append(val_loss)
        self.train_acc.append(tr_acc)
        self.test_acc.append(val_acc)

        print('ALL acc:', self.test_acc)

        if self.asym:
            file_name = 'log/asym_loss_%s_%s_%s.npy' % \
                        (self.model_name, self.dataset, self.noise_ratio)
            np.save(file_name, np.stack((np.array(self.train_loss), np.array(self.test_loss))))
            file_name = 'log/asym_acc_%s_%s_%s.npy' % \
                        (self.model_name, self.dataset, self.noise_ratio)
            np.save(file_name, np.stack((np.array(self.train_acc), np.array(self.test_acc))))
            file_name = 'log/asym_class_loss_%s_%s_%s.npy' % \
                        (self.model_name, self.dataset, self.noise_ratio)
            np.save(file_name, np.array(self.train_loss_class))
            file_name = 'log/asym_class_acc_%s_%s_%s.npy' % \
                        (self.model_name, self.dataset, self.noise_ratio)
            np.save(file_name, np.array(self.train_acc_class))
        else:
            file_name = 'log/loss_%s_%s_%s_%s.npy' % \
                        (self.model_name, self.dataset, self.noise_ratio, self.alpha)
            np.save(file_name, np.stack((np.array(self.train_loss), np.array(self.test_loss))))
            file_name = 'log/acc_%s_%s_%s_%s.npy' % \
                        (self.model_name, self.dataset, self.noise_ratio, self.alpha)
            np.save(file_name, np.stack((np.array(self.train_acc), np.array(self.test_acc))))

        return


class SGDLearningRateTracker(Callback):
    def __init__(self, model):
        super(SGDLearningRateTracker, self).__init__()
        self.model = model

    def on_epoch_begin(self, epoch, logs={}):
        init_lr = float(K.get_value(self.model.optimizer.lr))
        decay = float(K.get_value(self.model.optimizer.decay))
        iterations = float(K.get_value(self.model.optimizer.iterations))
        lr = init_lr * (1. / (1. + decay * iterations))
        print('init lr: %.4f, current lr: %.4f, decay: %.4f, iterations: %s' % (init_lr, lr, decay, iterations))

In [149]:
callbacks.append(SGDLearningRateTracker(model))

In [150]:
# acc, loss, lid
log_callback = LoggerCallback(model, X_train, y_tr, y_train_clean, X_test, y_test, dataset, model_name, noise_ratio, asym, epochs, alpha, beta)
callbacks.append(log_callback)

In [151]:
datagen = ImageDataGenerator()
datagen.fit(X_train)

In [ ]:
model.fit_generator(datagen.flow(X_train, y_tr, batch_size=batch_size),
                        steps_per_epoch=len(X_train) / batch_size, epochs=epochs,
                        validation_data=(X_test, y_test),
                        verbose=1,
                        callbacks=callbacks,
                         
                        )


In [153]:
loss, accuracy, f1_score, precision, recall,auc = model.evaluate(X_test, y_test, verbose=0)
print("Loss =",loss,"Accuracy =", accuracy,"f1 score =", f1_score,"Precission =", precision,"recall =", recall,"AUC score =",auc)

Loss = 1.008489966392517 Accuracy = 0.9717000126838684 f1 score = 0.9700540900230408 Precission = 0.9813153743743896 recall = 0.9594648480415344 AUC score = 0.9990072250366211
